In [1]:
from loca import print_date
print_date()

Last executed: 2019-08-19 16:40:08.558457 by jvano on casper21


In [2]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

import xarray as xr

from loca.data_catalog import load_daily_cmip_met_datasets
models = ['ACCESS1-0', 'CanESM2', 'CCSM4', 'CNRM-CM5', 'MIROC5', 'MRI-CGCM3', 'NorESM1-M']


In [3]:
from dask.distributed import Client
from dask_jobqueue import PBSCluster

In [4]:
cluster = PBSCluster(walltime='02:00:00')
cluster

In [5]:
cluster.scale(9)

In [6]:
client = Client(cluster)
client

Client Scheduler: tcp://10.148.10.19:43772 Dashboard: proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
def mean_max_precip(da):
    return da.resample(time='AS').max('time').mean('time')

def wet_day_fraction(da, thresh=0):
    nwet = da.where(da > thresh).count(dim="time")
    nday = da.sizes['time']
    return nwet / nday

In [31]:
def calc_and_write_stats(data, scen='hist'):
    if scen == 'hist':
        times = slice('1970', '1999')
    elif 'rcp' in scen:
        times = slice('2065', '2094')
    else:
        raise ValueError(period)
    try:
        da = data.sel(time=times).mean('time').load()
        print(f'/glade/u/home/jhamman/workdir/stats/mean_{method}_{scen}_{model}.nc')
        da.to_netcdf(f'/glade/u/home/jhamman/workdir/stats/mean_{method}_{scen}_{model}.nc')
    except:
        print('failed mean')
    
    try:
        da = mean_max_precip(data.sel(time=times)).load()
        da.to_netcdf(f'/glade/u/home/jhamman/workdir/stats/annmax_{method}_{scen}_{model}.nc')
    except:
        print('failed max')
    
    try:
        da = wet_day_fraction(data.sel(time=times)).load()
        da.to_netcdf(f'/glade/u/home/jhamman/workdir/stats/wetfrac_{method}_{scen}_{model}.nc')
    except:
        print('failed wdf')
        

In [9]:
hist_data = {}
rcp4_data = {}
rcp8_data = {}

In [13]:
for model in models:
    hist_data = load_daily_cmip_met_datasets('historical', models=[model], parallel=True)
    rcp4_data = load_daily_cmip_met_datasets('rcp45', models=[model], parallel=True)
    rcp8_data = load_daily_cmip_met_datasets('rcp85', models=[model], parallel=True)
    
    for method in ['loca', 'bcsd']:
        calc_and_write_stats(hist_data[method]['pcp'], scen='hist')
        calc_and_write_stats(rcp4_data[method]['pcp'], scen='rcp45')
        calc_and_write_stats(rcp8_data[method]['pcp'], scen='rcp85')

load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_dataset
load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_dataset
load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_dataset
/glade/u/home/jhamman/workdir/stats/mean_loca_hist_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_loca_rcp45_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_loca_rcp85_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_bcsd_hist_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_bcsd_rcp45_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_bcsd_rcp85_ACCESS1-0.nc
load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_dataset
load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_dataset
load_daily_cmip_met_datasets
load_daily_loca_meteorology
load_daily_bcsd_meteorology
load_bcsd_da

In [25]:
wrf_gcm_map = {'ACCESS1-0': 'access13', 'CanESM2': 'canesm', 'CCSM4': 'cesm', 'CNRM-CM5': 'cnrm', 'MIROC5': 'miroc5', 'MRI-CGCM3': 'mri', 'NorESM1-M': 'noresm'}

def load_daily_icar_datasets(scen, models=None, **kwargs):
    
    fname = '/glade/p/ral/hap/trude/conus_icar/qm_data/{gcm}_{scen}_exl_conv.nc'
    fnames = [fname.format(gcm=wrf_gcm_map[gcm], scen=scen) for gcm in models]
    ds = xr.open_mfdataset(fnames, **kwargs)
    
    return ds

configs = ['AR_p', 'AR_puv', 'AR_uvq', 'PA_p', 'PA_puv', 'PA_uvq', 'PR_p', 'PR_puv', 'PR_uvq', 'PT_p']
def load_daily_gard_gutmann_datasets(scen, models=None, configs=None, **kwargs):
    
    fname = '/glade/u/home/gutmann/work/gard/paper_data/processed/gard_{gcm}_{scen}_{config}.nc'
    
    ds_list = []
    for config in configs:
        fnames = [fname.format(gcm=wrf_gcm_map[gcm], scen=scen, config=config) for gcm in models]
        ds_list.append(xr.open_mfdataset(fnames, **kwargs))
        
    return xr.concat(ds_list, dim=xr.Variable('config', configs))
    
    

In [21]:
method = 'icar'
chunks = {'time': 183}
for model in models:
    hist_data[method] = load_daily_icar_datasets('hist', models=[model], chunks=chunks)
    rcp4_data[method] = load_daily_icar_datasets('rcp45', models=[model], chunks=chunks)
    rcp8_data[method] = load_daily_icar_datasets('rcp85', models=[model], chunks=chunks)
    
    calc_and_write_stats(hist_data[method]['pcp'], scen='hist')
    calc_and_write_stats(rcp4_data[method]['pcp'], scen='rcp45')
    calc_and_write_stats(rcp8_data[method]['pcp'], scen='rcp85')

/glade/u/home/jhamman/workdir/stats/mean_icar_hist_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp45_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp85_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_hist_CanESM2.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp45_CanESM2.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp85_CanESM2.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_hist_CCSM4.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp45_CCSM4.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp85_CCSM4.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_hist_CNRM-CM5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp45_CNRM-CM5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp85_CNRM-CM5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_hist_MIROC5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp45_MIROC5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_rcp85_MIROC5.nc
/glade/u/home/jhamman/workdir/stats/mean_icar_hist_MRI-CGCM3

In [35]:
wrf_gcm_map = {'ACCESS1-0': 'access13', 'CanESM2': 'canesm', 'CCSM4': 'ccsm4', 'CNRM-CM5': 'cnrm', 'MIROC5': 'miroc5', 'MRI-CGCM3': 'mri', 'NorESM1-M': 'noresm'}
gcms = ['ACCESS1-0', 'MIROC5', 'MRI-CGCM3', 'NorESM1-M', ]
# 'CNRM-CM5', 'CanESM2' # broken in GARD

method = 'gard'
chunks = {'time': 183}
for model in models:
    try:
        hist_data[method] = load_daily_gard_gutmann_datasets('hist', models=[model], configs=configs, chunks=chunks)
        rcp8_data[method] = load_daily_gard_gutmann_datasets('rcp85', models=[model], configs=configs, chunks=chunks)
    except FileNotFoundError:
        print('model failed')
        continue
    calc_and_write_stats(hist_data[method]['pcp'], scen='hist')
    calc_and_write_stats(rcp8_data[method]['pcp'], scen='rcp85')

/glade/u/home/jhamman/workdir/stats/mean_gard_hist_ACCESS1-0.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_ACCESS1-0.nc
failed mean
failed max
failed wdf
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_CanESM2.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_hist_CCSM4.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_CCSM4.nc
model failed
/glade/u/home/jhamman/workdir/stats/mean_gard_hist_MIROC5.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_MIROC5.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_hist_MRI-CGCM3.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_MRI-CGCM3.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_hist_NorESM1-M.nc
/glade/u/home/jhamman/workdir/stats/mean_gard_rcp85_NorESM1-M.nc


In [28]:
hist_data

{'icar': <xarray.Dataset>
 Dimensions:    (lat: 224, lon: 464, time: 20073)
 Coordinates:
   * time       (time) datetime64[ns] 1951-01-02T23:00:00 ... 2005-12-30T23:00:00
   * lat        (lat) float64 25.06 25.19 25.31 25.44 ... 52.56 52.69 52.81 52.94
   * lon        (lon) float64 -124.9 -124.8 -124.7 ... -67.31 -67.19 -67.06
 Data variables:
     pcp        (time, lat, lon) float32 dask.array<shape=(20073, 224, 464), chunksize=(183, 224, 464)>
     t_mean     (time, lat, lon) float32 dask.array<shape=(20073, 224, 464), chunksize=(183, 224, 464)>
     t_range    (time, lat, lon) float32 dask.array<shape=(20073, 224, 464), chunksize=(183, 224, 464)>
     elevation  (lat, lon) float64 dask.array<shape=(224, 464), chunksize=(224, 464)>
     mask       (lat, lon) int64 dask.array<shape=(224, 464), chunksize=(224, 464)>
     t_max      (time, lat, lon) float32 dask.array<shape=(20073, 224, 464), chunksize=(183, 224, 464)>
     t_min      (time, lat, lon) float32 dask.array<shape=(20073, 2